In [ ]:
import numpy as np
from scipy.linalg import sqrtm
import json
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
# the bases we need to measure the state in, 81=3^4 in total
# the bases go from XXXX to ZZZZ in lexicographical order
bases = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                bases.append([i, j, k, l])

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 7 qubit topology
layouts_7qubit = [
    [1,3,0,5], 
    [3,5,1,4],
    [1,3,2,5],
    [3,5,1,6],
    [3,1,5,0],
    [5,3,4,1],
    [3,1,5,2],
    [5,3,6,1]
          ]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit linear topology
layouts_5qubit_linear = [
    [1,2,0,3],
    [2,1,3,0],
    [2,3,1,4],
    [3,2,4,1]
]

In [ ]:
# layouts that achieve a 9 CNOT circuit on the 5 qubit T-shape topology
layouts_5qubit_Tshape = [
    [1,3,0,4],
    [3,1,4,0],
    [1,3,2,4],
    [3,1,4,2]
]

In [ ]:
# definition of basis measurement gates and pauli basis gates, needed for density matrix reconstruction
I = np.array([[1, 0], [0, 1]])
H = (1/np.sqrt(2))*np.array([[1, 1], [1, -1]])
HSi = (1/np.sqrt(2))*np.array([[1, -1j], [1, 1j]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
# ideal Cabello state
state = (1/np.sqrt(3))*np.array([0, 0, 0, 1, 0, -0.5, -0.5, 0, 0, -0.5, -0.5, 0, 1, 0, 0, 0])
# list basis gates for tomography measurement and pauli basis gates, 
tomo_gates = [H, HSi, I, I]
pauli_gates = [X, Y, Z, I]

In [ ]:
# extra bases, the bases containing the identity operator. we don't need to measure these
# basis states here are of the form [i, j, k, l] with i, j, k, l going from 0 to 3, 
# 0 = X, 1 = Y, 2 = Z and 3 = I
# and at least one of i, j, k, or l is 3
extra_bases = []
extra_bases.append([3, 3, 3, 3])
for i in range(3):
    extra_bases.append([i, 3, 3, 3])
    extra_bases.append([3, i, 3, 3])
    extra_bases.append([3, 3, i, 3])
    extra_bases.append([3, 3, 3, i])
    for j in range(3):
        extra_bases.append([3, 3, i, j])
        extra_bases.append([3, i, 3, j])
        extra_bases.append([3, i, j, 3])
        extra_bases.append([i, 3, 3, j])
        extra_bases.append([i, 3, j, 3])
        extra_bases.append([i, j, 3, 3])
        for k in range(3):
                extra_bases.append([i, j, k, 3])
                extra_bases.append([i, j, 3, k])
                extra_bases.append([i, 3, j, k])
                extra_bases.append([3, i, j, k])

In [ ]:
def classical_fidelity(quasi_dists, basis, num_run):
    '''calculates the classical fidelity of the measured result
    with respect to the ideal cabello state in the chosen basis'''
    gates = [H, HSi, I]
    gate = np.kron(np.kron(np.kron(gates[basis[0]], gates[basis[1]]), gates[basis[2]]), gates[basis[3]])
    final_state = gate.dot(state)
    perf_probs = {f'{k:04b}': abs(final_state[k])**2 for k in range(16)}
    meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[num_run].items()}
    fidelity = 0
    for bitstring in perf_probs:
        fidelity += np.sqrt(perf_probs[bitstring]*meas_probs.get(bitstring, 0))
    return fidelity**2

In [ ]:
def mitigate(probs, T):
    '''create readout error mitigated probability distribution
    from the raw measurement probability distribution and the readout error transition matrix
    parameters:
    ----------
    probs: unmitigated probability distribution
    T: readout error transition matrix
    
    output:
    error mitigated probability distribution.
    '''
    ps = list(probs.values())
    mitig_probs = np.linalg.inv(T).dot(ps)
    return {key: mitig_probs[i] for i, key in enumerate(probs.keys())}

In [ ]:
def classical_fidelity_mitig(quasi_dists, basis, num_run):
    '''calculates the classical fidelity of the error mitigated result
    with respect to the ideal cabello state in the chosen basis'''
    gates = [H, HSi, I]
    gate = np.kron(np.kron(np.kron(gates[basis[0]], gates[basis[1]]), gates[basis[2]]), gates[basis[3]])
    final_state = gate.dot(state)
    perf_probs = {f'{k:04b}': abs(final_state[k])**2 for k in range(16)}
    meas_probs = {f'{bitstring:04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[num_run].items()}
    mitig_probs = mitigate(meas_probs, T)
    fidelity = 0
    for bitstring in perf_probs:
        fidelity += np.sqrt(perf_probs[bitstring]*mitig_probs.get(bitstring, 0))
    return fidelity**2

In [ ]:
def get_full_bitstring(bitstring, layout, n):
    bs = '0'*n
    for i, qubit in enumerate(layout):
        bs = bs[:qubit] + bitstring[i] + bs[qubit+1:]
    return bs[::-1]

In [ ]:
def calc_T(prob_dists, layout, n):
    T = np.zeros((16, 16))
    for i in range(16):
        prob_dist = prob_dists[i]
        for j in range(16):
            bitstring = f'{j:04b}'
            for (bs, prob) in prob_dist.items():
                bs = f'{int(bs):07b}'[::-1]
                if ((bs[layout[0]] == bitstring[0]) and (bs[layout[1]] == bitstring[1])
                    and (bs[layout[2]] == bitstring[2]) and (bs[layout[3]] == bitstring[3])):
                    T[i, j] += prob
    return T

In [ ]:
# calculates the p(1->0) readout error from the error circuit measurements for the chosen qubit
def calc_p10(prob_dist, qubit):
    p = 0
    for (bs, prob) in prob_dist.items():
        if f'{int(bs):07b}'[::-1][qubit] == '0':
            p += prob
    return p

In [ ]:
# calculates the p(0->1) readout error from the error circuit measurements for the chosen qubit
def calc_p01(prob_dists, qubit):
    p = 0
    for prob_dist in prob_dists:
        for (bs, prob) in prob_dist.items():
            if f'{int(bs):07b}'[::-1][qubit] == '1':
                p += prob
    return p / len(prob_dists)

In [ ]:
def density_matrix(quasi_dists):
    '''
    Reconstructing the density matrix from the measurement results.
    The equation used (\sigma are the four Pauli matrices): 
    \rho = (1/16)*sum_{ijkl=0}^{3} {\sigma_i \sigma_j \sigma_k \sigma_l \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho}},
    where \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho} is estimated from the measurement results.
    parameters:
    ----------
    quasi_dists: the measured distributions, ordered by basis
    
    output: 
    the reconstructed density matrix.
    '''
    rho = np.zeros((16, 16), dtype='complex128')
    for (basis, num_run) in zip(bases, range(len(bases))):
        meas_probs = {f'{int(bitstring):04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[num_run].items()}
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = sum((-1)**(state.count('1') % 2)*meas_probs[state] for state in meas_probs)
        rho += trace_rho * pauli_matrix
    for basis in extra_bases:
        og_basis = [b if b!=3 else 2 for b in basis]
        no = og_basis[0]*27+og_basis[1]*9+og_basis[2]*3+og_basis[3]
        meas_probs = {f'{int(bitstring):04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[no].items()}
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = 0
        for curr_state in meas_probs:
            new_state = curr_state
            for i in range(len(basis)):
                if basis[i] == 3:
                    new_state = new_state[:i] + '0' + new_state[i+1:]
            trace_rho += (-1)**(new_state.count('1') % 2)*meas_probs[curr_state]
        rho += trace_rho * pauli_matrix
    return (1/16)*rho

In [ ]:
def density_matrix_mitig(quasi_dists, T):
    '''
    Reconstructing the density matrix from the measurement results.
    The equation used (\sigma are the four Pauli matrices): 
    \rho = (1/16)*sum_{ijkl=0}^{3} {\sigma_i \sigma_j \sigma_k \sigma_l \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho}},
    where \Tr{\sigma_i \sigma_j \sigma_k \sigma_l \rho} is estimated from the measurement results
    and mitigated using simple inversion assuming a tensor product matrix readout error noise model.
    parameters:
    ----------
    quasi_dists: the measured distributions, ordered by basis
    T: readout error transition matrix
    
    output:
    the reconstructed density matrix.
    '''
    rho = np.zeros((16, 16), dtype='complex128')
    for (basis, num_run) in zip(bases, range(len(bases))):
        meas_probs = {f'{int(bitstring):04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[num_run].items()}
        mitig_probs = mitigate(meas_probs, T)
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = sum((-1)**(state.count('1') % 2)*mitig_probs[state] for state in mitig_probs)
        rho += trace_rho * pauli_matrix
    for basis in extra_bases:
        og_basis = [b if b!=3 else 2 for b in basis]
        no = og_basis[0]*27+og_basis[1]*9+og_basis[2]*3+og_basis[3]
        meas_probs = {f'{int(bitstring):04b}'[::-1]: prob for (bitstring, prob) in quasi_dists[no].items()}
        mitig_probs = mitigate(meas_probs, T)
        pauli_matrix = np.kron(np.kron(np.kron(pauli_gates[basis[0]], pauli_gates[basis[1]]),
                                       pauli_gates[basis[2]]), pauli_gates[basis[3]])
        trace_rho = 0
        for curr_state in mitig_probs:
            new_state = curr_state
            for i in range(len(basis)):
                if basis[i] == 3:
                    new_state = new_state[:i] + '0' + new_state[i+1:]
            trace_rho += (-1)**(new_state.count('1') % 2)*mitig_probs[curr_state]
        rho += trace_rho * pauli_matrix
    return (1/16)*rho

In [ ]:
def quantum_fidelity(rho1, rho2):
    '''Function to calculate the quantum fidelity between two density matrices rho1 and rho2'''
    return (np.trace(sqrtm(np.matmul(np.matmul(sqrtm(rho1), rho2), sqrtm(rho1)))))**2

In [ ]:
def calculate_QST_quantum_fidelity(filename):
    '''calculates raw and mitigated quantum fidelity of Cabello state tomography measurement result.
    parameters:
    ----------
    job_id: id of the qiskit job to evaluate
    layout_num: layout of the qubits used in the job
    
    output:
    quantum fidelity of the unmitigated and the mitigated state tomography measurement result'''
    with open(filename, 'r') as f:
        data = f.readlines()
        backend_name = data[0][:-1]
        layout = json.loads(data[1][:-1])
        if len(data) == 3:
            quasi_dists = json.loads(data[2])
        else:
            quasi_dists = json.loads(data[3])
    backend = service.get_backend(backend_name)
    n = backend.configuration().n_qubits
    readout_meas = quasi_dists[81:]
    T = calc_T(readout_meas, layout, n)
    rho_meas = density_matrix(quasi_dists) # unmitigated density matrix
    rho_mitig = density_matrix_mitig(quasi_dists, T) # mitigated density matrix
    rho_ideal = np.outer(state, state)
    return quantum_fidelity(rho_meas, rho_ideal).real, quantum_fidelity(rho_mitig, rho_ideal).real